##Práctica 3. Extacción de palabras clave y resumen automático.

In [9]:
#Importar librerias
import nltk
import re
import spacy
from transformers import BertTokenizer, BertModel
import torch
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
import string
from rake_nltk import Rake
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer

In [10]:
#Cargar los documentos.
with open('libros\The Adventures of Sherlock Holmes.txt' ,'r', encoding = 'utf-8') as archivo:
    libro1 = archivo.read()

with open('libros\The Last Man.txt' ,'r', encoding = 'utf-8') as archivo:
    libro2 = archivo.read()

with open('libros\Frankenstein.txt' ,'r', encoding = 'utf-8') as archivo:
    libro3 = archivo.read()

with open('libros\The War of the Worlds.txt' ,'r', encoding = 'utf-8') as archivo:
    libro4 = archivo.read()

with open('libros\THE SECRET ADVERSARY.txt' ,'r', encoding = 'utf-8') as archivo:
    libro5 = archivo.read()

with open('libros\The Mysterious Affair at Styles.txt' ,'r', encoding = 'utf-8') as archivo:
    libro6 = archivo.read()


<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:5: SyntaxWarning: invalid escape sequence '\T'
<>:8: SyntaxWarning: invalid escape sequence '\F'
<>:11: SyntaxWarning: invalid escape sequence '\T'
<>:14: SyntaxWarning: invalid escape sequence '\T'
<>:17: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:5: SyntaxWarning: invalid escape sequence '\T'
<>:8: SyntaxWarning: invalid escape sequence '\F'
<>:11: SyntaxWarning: invalid escape sequence '\T'
<>:14: SyntaxWarning: invalid escape sequence '\T'
<>:17: SyntaxWarning: invalid escape sequence '\T'
C:\Users\davic\AppData\Local\Temp\ipykernel_16272\2199434100.py:2: SyntaxWarning: invalid escape sequence '\T'
  with open('libros\The Adventures of Sherlock Holmes.txt' ,'r', encoding = 'utf-8') as archivo:
C:\Users\davic\AppData\Local\Temp\ipykernel_16272\2199434100.py:5: SyntaxWarning: invalid escape sequence '\T'
  with open('libros\The Last Man.txt' ,'r', encoding = 'utf-8') as archivo:

#Frases más representativas con  TF-IDF

In [11]:
def top_Words(texto):
    # Convertir a minúsculas y eliminar puntos
    texto = texto.lower().replace('.', '') 
    #Tokenizar el texto
    tokens = nltk.word_tokenize(texto)
    #aplicar pos tagging
    tagged = nltk.pos_tag(tokens)
    #Obtener solo sustantivos
    nouns = [word for (word, tag) in tagged if tag.startswith("NN")]
    #Aplicar tf-idf
    #En lugar de pasar el texto completo, se pasa solo los sustantivos
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform([" ".join(nouns)])
    #Obtener las palabras mas importantes
    top_words = sorted( vectorizer.vocabulary_, key = lambda x: X[0,vectorizer.vocabulary_[x]], reverse= True)[:5]
    #imprimir las palabras mas importantes
    print(f"{top_words}\n")

#Remover indiceS
libro1 = re.sub(r'Contents.*?(?=I.\n)', '', libro1, flags=re.DOTALL)
libro2 = re.sub(r'Contents.*?(?=I visited)', '', libro2, flags=re.DOTALL)
libro3 = re.sub(r'CONTENTS.*?(?=England._\n)', '', libro3, flags=re.DOTALL)
libro4 = re.sub(r'But.*?(?=No one would)', '', libro4, flags=re.DOTALL)
libro5 = re.sub(r'CONTENTS.*?AND AFTER', '', libro5, flags=re.DOTALL)
libro6 = re.sub(r'Contents.*?(?=EXPLAINS\n)', '', libro6, flags=re.DOTALL)

top_Words(libro1)
top_Words(libro2)
top_Words(libro3)
top_Words(libro4)
top_Words(libro5)
top_Words(libro6)

['holmes', 'man', 'room', 'time', 'door']

['raymond', 'life', 'heart', 'time', 'man']

['man', 'father', 'life', 'eyes', 'time']

['martians', 'people', 'man', 'time', 'men']

['tuppence', 'tommy', 'julius', 'james', 'man']

['poirot', 'inglethorp', 'john', 'room', 'mrs']



#Frases más representativas con rake

In [12]:
rake_nltk_var = Rake()

def top_Words_rake(texto):
    #Remover caracteres especiales
    texto = re.sub(r'[^\w\s]', '', texto)
    #Remover saltos de linea
    texto = texto.replace('\n', ' ')
    #Extraer las palabras clave
    rake_nltk_var.extract_keywords_from_text(texto)
    keyword_extracted = rake_nltk_var.get_ranked_phrases()
    #Imprimir las palabras clave
    print(f"\n\n {keyword_extracted[:5]}")

top_Words_rake(libro1)
top_Words_rake(libro2)
top_Words_rake(libro3)
top_Words_rake(libro4)
top_Words_rake(libro5)
top_Words_rake(libro6)




 ['said lestrade oct 4th rooms 8_s_ breakfast 2_s_ 6_d_ cocktail 1_s_ lunch 2_s_ 6_d_ glass sherry 8_d_', '_globe_ _star_ _pall mall_ _st jamess gazette_ _evening news_ _standard_ _echo_', 'year 1858 contraltohum la scala hum prima donna imperial opera', 'st augustine 9th mccauley cleared 10th john swain cleared 12th visited paramore', 'new zealand stock paying 4½ per cent two thousand five hundred pounds']


 ['di mie tenere frondi altro lavoro credea mostrarte e qual fero pianeta ne nvidiò insieme', 'cada flor costruye un monumento cada edificio es un sepulcro altivo cada soldado un esqueleto vivo3', 'human race 10 calderon de la barca 11 2 wordsworth 12 keats 13 andrew marvell 14', 'secluded life pareamo aver qui tutto il ben raccolto che fra mortali', 'obsequie18 16 wordsworth 17 priors solomon 18 clevelands poems vol iii chapter']


 ['affectionate brother r walton letter 2 _to mrs saville england_ archangel 28th march 17', 'affectionate brother robert walton letter 3 _to mrs sa

#Frases mas representativas con BERT

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def top_Words_bert(texto):
    tokens = tokenizer.encode(texto, add_special_tokens=True, return_tensors='pt')

    # Dividir el texto en segmentos más pequeños
    max_segment_len = 512 
    num_segments = (tokens.size(1) - 2) // max_segment_len + 1
    segment_embeddings = []

    for i in range(num_segments):
        start_idx = i * max_segment_len
        end_idx = min((i + 1) * max_segment_len, tokens.size(1))
        segment_tokens = tokens[:, start_idx:end_idx]

        # Obtener las representaciones de palabras contextualizadas
        with torch.no_grad():
            outputs = model(segment_tokens)
            word_embeddings = outputs.last_hidden_state

        # Promediar las representaciones de palabras para obtener una representación de segmento
        segment_embedding = torch.mean(word_embeddings, dim=1).squeeze()
        segment_embeddings.append(segment_embedding)

    # Concatenar las representaciones de segmento y promediarlas para obtener una representación de oración global
    sentence_embedding = torch.mean(torch.stack(segment_embeddings), dim=0)

    top_words = []
    for word_idx in torch.topk(sentence_embedding, 10).indices:
        word = tokenizer.decode([tokens[0, word_idx]])
        if not word.startswith('#' or '...' or '!' or ';'):
            top_words.append(word)
    #Imprimir las  5 palabras mas importantes 
    print(f"\n\n {top_words[:5]}")
    


#Funcion para remover signos de puntuacion y stopwords
def remove_punctuation_stopwords(text):
    # Remover signos de puntuación
    patron = re.compile(r'[#,\-_.;:‘’!¿?"”]+')
    text = patron.sub('', text)
    # Remover stopwords
    text = ' '.join([word for word in text.split() if word.lower() not in STOP_WORDS])
    top_Words_bert(text)

remove_punctuation_stopwords(libro1)
remove_punctuation_stopwords(libro2)
remove_punctuation_stopwords(libro3)
remove_punctuation_stopwords(libro4)
remove_punctuation_stopwords(libro5)
remove_punctuation_stopwords(libro6)

c:\Users\davic\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (51210 > 512). Running this sequence through the model will result in indexing errors




 ['signs', 'experiences', 'powers', 'royal', 'sex']


 ['preservation', 'persisted', 'companion', 'return', 'surrounding']


 ['composed', 'march', 'dec', 'tread', 'di']


 ['heat', 'men', 'ceased', 'un', 'bison']


 ['chance', 'hear', 'don', 'know', 'pm']


 ['lawrence', 'came', 'possessed', 'subsided', 'regularly']


#Frases más representativas con Textrank

In [14]:
# Cargar el modelo de idioma
nlp = spacy.load("en_core_web_sm")

# Función para extraer frases clave usando TextRank
def top_words_spacy(text, max_phrase_length=10):
    # Procesar el texto con spaCy
    doc = nlp(text)
    
    # Construir una lista de oraciones
    sentences = [sent.text for sent in doc.sents]
    
    # Calcular el puntaje de TextRank para cada oración
    wordsSpacy = []
    for sent in sentences:
        phrase_doc = nlp(sent)
        phrase_tokens = [token.text for token in phrase_doc if not token.is_stop and not token.is_punct]
        if len(phrase_tokens) <= max_phrase_length:
            wordsSpacy.append((sent, len(phrase_tokens)))
    
    # Ordenar las frases clave por puntaje de TextRank
    wordsSpacy = sorted(wordsSpacy, key=lambda x: x[1], reverse=True)
    #Imprimir las frases clave
    print("[")
    for frase, _ in wordsSpacy[:5]:
        print(f"\"{frase}\" ,\n")
    print("\n]\n")


top_words_spacy(libro1)
top_words_spacy(libro2)
top_words_spacy(libro3)
top_words_spacy(libro4)
top_words_spacy(libro5)
top_words_spacy(libro6)


[
"They were admirable things for the observer—excellent for
drawing the veil from men’s motives and actions." ,

"He was pacing the room swiftly, eagerly, with his
head sunk upon his chest and his hands clasped behind him." ,

"What do you deduce from
it?”

I carefully examined the writing, and the paper upon which it was
written.

" ,

"His eyes sparkled, and he sent up a great blue triumphant cloud
from his cigarette.

" ,

"A man entered who could hardly have been less than six feet six inches
in height, with the chest and limbs of a Hercules." ,


]

[
"We passed
by a natural archway, leading to a second gallery, and enquired, if we
could not enter there also." ,

"Shall I go alone, or will you accompany me?”

I signified my readiness to proceed, but our guides protested against
such a measure." ,

"On
examination, we found that all the leaves, bark, and other substances,
were traced with written characters." ,

"Scattered and unconnected as they were, I have been obliged to
add l

#Resumen con Frecuencia de palabras.

In [16]:
def resumenF(texto):
    doc = nlp(texto)

    tokens = [token.text for token in doc]

    # Eliminar palabras vacías y signos de puntuación
    word_frequencies = {}
    for word in doc:
        if word.text.lower() not in stopwords:
            if word.text.lower() not in string.punctuation:
                # Contar las frecuencias de las palabras
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1

    max_frequency = max(word_frequencies.values())

    # Normalizar las frecuencias de las palabras
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word]/max_frequency

    sentence_list = [ sentence for sentence in doc.sents ]

    # Calcular los puntajes de las frases
    sentence_scores = {}
    for sent in sentence_list:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]


    # Ordenar las frases según sus puntajes de evaluación
    top_sentences = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)[:10]

    # Concatenar las frases seleccionadas en un solo párrafo
    summary = " ".join([sentence.text.strip() for sentence, score in top_sentences])

    # Imprimir el resumen
    print("Resumen:")
    print(summary)


#Cargar los libros para los resumenes
with open('libros/MOBY-DICK;.txt' ,'r', encoding = 'utf-8') as archivo:
    libro7 = archivo.read()

with open('libros/Alice’s Adventures in Wonderland.txt', 'r', encoding = 'utf-8') as archivo:
    libro8 = archivo.read()


stopwords = list(STOP_WORDS)

nlp = spacy.load('en_core_web_sm')

#Limpiar los libros (remover indices y recuperar el primer capitulo.)
libro7 = re.sub(r'CONTENTS.*?(?=Call me Ishmael)', '', libro7, flags=re.DOTALL)
patron = re.compile(r"Call me Ishmael(.*?)CHAPTER 2", re.DOTALL)
resultado = patron.search(libro7)

libro8 = re.sub(r'THE MILLENNIUM FULCRUM EDITION 3\.0.*?(?=Alice was)', '', libro8, flags= re.DOTALL)
patron = re.compile(r"Alice was.*?CHAPTER II", re.DOTALL)
resultado2 = patron.search(libro8)

resumenF(resultado.group(0))
print("\n")
resumenF(resultado2.group(0))


Resumen:
Go visit the Prairies in June,
when for scores on scores of miles you wade knee-deep among
Tiger-lilies—what is the one charm wanting?—Water—there is not a drop
of water there! And more than all, if
just previous to putting your hand into the tar-pot, you have been
lording it as a country schoolmaster, making the tallest boys stand in
awe of you. Its extreme
downtown is the battery, where that noble mole is washed by waves, and
cooled by breezes, which a few hours previous were out of sight of
land. No, when I go to sea, I go as a simple sailor, right before the mast,
plumb down into the forecastle, aloft there to the royal mast-head. There now is your insular city of the Manhattoes, belted round by
wharves as Indian isles by coral reefs—commerce surrounds it with her
surf. But these
are all landsmen; of week days pent up in lath and plaster—tied to
counters, nailed to benches, clinched to desks. Why is almost every robust healthy boy with a
robust healthy soul in him, at some

#Resumen con Textrank

In [17]:
def text_rank(text, num_sentences=10):
    # Tokenizar el texto
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    
    summarizer = TextRankSummarizer()
    
    # Obtener las calificaciones de las oraciones
    ratings = summarizer.rate_sentences(parser.document)
    
    # Ordenar las oraciones en función de sus calificaciones
    sorted_sentences = sorted(ratings, key=lambda x: -ratings[x])
    
    # Seleccionar las mejores oraciones
    top_sentences = sorted_sentences[:num_sentences]

    # Imprimir las mejores oraciones como lista
    """print("Mejores oraciones:")
    for i, sentence in enumerate(top_sentences, 1):
        print(f"{i}. {sentence}")"""
    
    # Construir el resumen con las mejores oraciones
    summarized_text = " ".join(str(sentence) for sentence in top_sentences)
    
    # Imprimir el resumen
    print("Resumen:")
    print(summarized_text)


text_rank(resultado.group(0))
print("\n")
text_rank(resultado2.group(0))



Resumen:
Though I cannot tell why it was exactly that those stage managers, the Fates, put me down for this shabby part of a whaling voyage, when others were set down for magnificent parts in high tragedies, and short and easy parts in genteel comedies, and jolly parts in farces—though I cannot tell why this was exactly; yet, now that I recall all the circumstances, I think I can see a little into the springs and motives which being cunningly presented to me under various disguises, induced me to set about performing the part I did, besides cajoling me into the delusion that it was a choice resulting from my own unbiased freewill and discriminating judgment. By reason of these things, then, the whaling voyage was welcome; the great flood-gates of the wonder-world swung open, and in the wild conceits that swayed me to my purpose, two and two there floated into my inmost soul, endless processions of the whale, and, mid most of them all, one grand hooded phantom, like a snow hill in the a

#Resumen con LSA

In [18]:
def lsa_summarizer(text, num_sentences=10):
    # Tokenizar el texto
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    
    summarizer = LsaSummarizer()

    # Crear el resumen
    summary = summarizer(parser.document, num_sentences)
    # Imprimir el resumen
    print("Resumen:")
    for sentence in summary:
        print(sentence) 
    


lsa_summarizer(resultado.group(0))
print("\n")
lsa_summarizer(resultado2.group(0))

Resumen:
Some years ago—never mind how long precisely—having little or no money in my purse, and nothing particular to interest me on shore, I thought I would sail about a little and see the watery part of the world.
With a philosophical flourish Cato throws himself upon his sword; I quietly take to the ship.
Deep into distant woodlands winds a mazy way, reaching to overlapping spurs of mountains bathed in their hill-side blue.
Were Niagara but a cataract of sand, would you travel your thousand miles to see it?
Why did the poor poet of Tennessee, upon suddenly receiving two handfuls of silver, deliberate whether to buy him a coat, which he sadly needed, or invest his money in a pedestrian trip to Rockaway Beach?
For my part, I abominate all honorable respectable toils, trials, and tribulations of every kind whatsoever.
And as for going as cook,—though I confess there is considerable glory in that, a cook being a sort of officer on ship-board—yet, somehow, I never fancied broiling fowls